In [3]:
import sys
sys.path.append('/home/dong/Dropbox/Projects/NLP/DeepQA')

In [15]:
from chatbot.chatbot import Chatbot

In [6]:
import tensorflow as tf
import numpy as np

from datasets.twitter import data
import data_utils

# load data from pickle and npy files
metadata, idx_q, idx_a = data.load_data(PATH='datasets/twitter/')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)

# parameters
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 4
xvocab_size = len(metadata['idx2w'])
yvocab_size = xvocab_size
emb_dim = 1024

In [21]:
print('xvocab_size: %s' % xvocab_size)
print(trainX.shape)

xvocab_size: 6002
(187262, 20)


In [ ]:
def buildNetwork(self):
    """ Create the computational graph
    """

    # TODO: Create name_scopes (for better graph visualisation)
    # TODO: Use buckets (better perfs)

    # Parameters of sampled softmax (needed for attention mechanism and a large vocabulary size)
    outputProjection = None
    # Sampled softmax only makes sense if we sample less than vocabulary size.
    if 0 < self.args.softmaxSamples < self.textData.getVocabularySize():
        outputProjection = ProjectionOp(
            (self.textData.getVocabularySize(), self.args.hiddenSize),
            scope='softmax_projection',
            dtype=self.dtype
        )

        def sampledSoftmax(labels, inputs):
            labels = tf.reshape(labels, [-1, 1])  # Add one dimension (nb of true classes, here 1)

            # We need to compute the sampled_softmax_loss using 32bit floats to
            # avoid numerical instabilities.
            localWt     = tf.cast(outputProjection.W_t,             tf.float32)
            localB      = tf.cast(outputProjection.b,               tf.float32)
            localInputs = tf.cast(inputs,                           tf.float32)

            return tf.cast(
                tf.nn.sampled_softmax_loss(
                    localWt,  # Should have shape [num_classes, dim]
                    localB,
                    labels,
                    localInputs,
                    self.args.softmaxSamples,  # The number of classes to randomly sample per batch
                    self.textData.getVocabularySize()),  # The number of classes
                self.dtype)

    # Creation of the rnn cell
    def create_rnn_cell():
        encoDecoCell = tf.contrib.rnn.BasicLSTMCell(  # Or GRUCell, LSTMCell(args.hiddenSize)
            self.args.hiddenSize,
        )
        if not self.args.test:  # TODO: Should use a placeholder instead
            encoDecoCell = tf.contrib.rnn.DropoutWrapper(
                encoDecoCell,
                input_keep_prob=1.0,
                output_keep_prob=self.args.dropout
            )
        return encoDecoCell
    encoDecoCell = tf.contrib.rnn.MultiRNNCell(
        [create_rnn_cell() for _ in range(self.args.numLayers)],
    )

    # Network input (placeholders)

    with tf.name_scope('placeholder_encoder'):
        self.encoderInputs  = [tf.placeholder(tf.int32,   [None, ]) for _ in range(self.args.maxLengthEnco)]  # Batch size * sequence length * input dim

    with tf.name_scope('placeholder_decoder'):
        self.decoderInputs  = [tf.placeholder(tf.int32,   [None, ], name='inputs') for _ in range(self.args.maxLengthDeco)]  # Same sentence length for input and output (Right ?)
        self.decoderTargets = [tf.placeholder(tf.int32,   [None, ], name='targets') for _ in range(self.args.maxLengthDeco)]
        self.decoderWeights = [tf.placeholder(tf.float32, [None, ], name='weights') for _ in range(self.args.maxLengthDeco)]

    # Define the network
    # Here we use an embedding model, it takes integer as input and convert them into word vector for
    # better word representation
    decoderOutputs, states = tf.contrib.legacy_seq2seq.embedding_rnn_seq2seq(
        self.encoderInputs,  # List<[batch=?, inputDim=1]>, list of size args.maxLength
        self.decoderInputs,  # For training, we force the correct output (feed_previous=False)
        encoDecoCell,
        self.textData.getVocabularySize(),
        self.textData.getVocabularySize(),  # Both encoder and decoder have the same number of class
        embedding_size=self.args.embeddingSize,  # Dimension of each word
        output_projection=outputProjection.getWeights() if outputProjection else None,
        feed_previous=bool(self.args.test)  # When we test (self.args.test), we use previous output as next input (feed_previous)
    )

    # TODO: When the LSTM hidden size is too big, we should project the LSTM output into a smaller space (4086 => 2046): Should speed up
    # training and reduce memory usage. Other solution, use sampling softmax

    # For testing only
    if self.args.test:
        if not outputProjection:
            self.outputs = decoderOutputs
        else:
            self.outputs = [outputProjection(output) for output in decoderOutputs]

        # TODO: Attach a summary to visualize the output

    # For training only
    else:
        # Finally, we define the loss function
        self.lossFct = tf.contrib.legacy_seq2seq.sequence_loss(
            decoderOutputs,
            self.decoderTargets,
            self.decoderWeights,
            self.textData.getVocabularySize(),
            softmax_loss_function= sampledSoftmax if outputProjection else None  # If None, use default SoftMax
        )
        tf.summary.scalar('loss', self.lossFct)  # Keep track of the cost

        # Initialize the optimizer
        opt = tf.train.AdamOptimizer(
            learning_rate=self.args.learningRate,
            beta1=0.9,
            beta2=0.999,
            epsilon=1e-08
        )
        self.optOp = opt.minimize(self.lossFct)

In [17]:
args_in = '--device gpu --modelTag '\
'v11_2l_word2vec_lr002_dr09 --learningRate 0.002 '\
'--dropout 0.9 --rootDir /home/dong/Dropbox/Projects/NLP/DeepQA '\
'--initEmbeddings'.split()

args = Chatbot.parseArgs(args_in)
print(repr(args))

Namespace(autoEncode=False, batchSize=32, corpus='ubuntu', createDataset=False, datasetTag='', debug=False, device='gpu', dropout=0.9, embeddingSize=256, embeddingSource='GoogleNews-vectors-negative300.bin', filterVocab=1, hiddenSize=256, initEmbeddings=True, keepAll=False, learningRate=0.002, maxLength=10, modelTag='v11_2l_word2vec_lr002_dr09', numEpochs=50, numLayers=2, playDataset=None, ratioDataset=1.0, reset=False, rootDir='/home/dong/Dropbox/Projects/NLP/DeepQA', saveEvery=2000, seed=None, skipLines=False, softmaxSamples=0, test=None, verbose=False, vocabularySize=40000, watsonMode=False)


In [ ]:
class Model:
    """
    Implementation of a seq2seq model.
    Architecture:
        Encoder/decoder
        2 LTSM layers
    """

    def __init__(self, args, textData):
        """
        Args:
            args: parameters of the model
            textData: the dataset object
        """
        print("Model creation...")

        self.textData = textData  # Keep a reference on the dataset
        self.args = args  # Keep track of the parameters of the model
        self.dtype = tf.float32

        # Placeholders
        self.encoderInputs  = None
        self.decoderInputs  = None  # Same that decoderTarget plus the <go>
        self.decoderTargets = None
        self.decoderWeights = None  # Adjust the learning to the target sentence size

        # Main operators
        self.lossFct = None
        self.optOp = None
        self.outputs = None  # Outputs of the network, list of probability for each words

        # Construct the graphs
        self.buildNetwork()

    def buildNetwork(self):
        """ Create the computational graph
        """

        # TODO: Create name_scopes (for better graph visualisation)
        # TODO: Use buckets (better perfs)

        # Parameters of sampled softmax (needed for attention mechanism and a large vocabulary size)
        outputProjection = None
        # Sampled softmax only makes sense if we sample less than vocabulary size.
        if 0 < self.args.softmaxSamples < self.textData.getVocabularySize():
            outputProjection = ProjectionOp(
                (self.textData.getVocabularySize(), self.args.hiddenSize),
                scope='softmax_projection',
                dtype=self.dtype
            )

            def sampledSoftmax(labels, inputs):
                labels = tf.reshape(labels, [-1, 1])  # Add one dimension (nb of true classes, here 1)

                # We need to compute the sampled_softmax_loss using 32bit floats to
                # avoid numerical instabilities.
                localWt     = tf.cast(outputProjection.W_t,             tf.float32)
                localB      = tf.cast(outputProjection.b,               tf.float32)
                localInputs = tf.cast(inputs,                           tf.float32)

                return tf.cast(
                    tf.nn.sampled_softmax_loss(
                        localWt,  # Should have shape [num_classes, dim]
                        localB,
                        labels,
                        localInputs,
                        self.args.softmaxSamples,  # The number of classes to randomly sample per batch
                        self.textData.getVocabularySize()),  # The number of classes
                    self.dtype)

        # Creation of the rnn cell
        def create_rnn_cell():
            encoDecoCell = tf.contrib.rnn.BasicLSTMCell(  # Or GRUCell, LSTMCell(args.hiddenSize)
                self.args.hiddenSize,
            )
            if not self.args.test:  # TODO: Should use a placeholder instead
                encoDecoCell = tf.contrib.rnn.DropoutWrapper(
                    encoDecoCell,
                    input_keep_prob=1.0,
                    output_keep_prob=self.args.dropout
                )
            return encoDecoCell
        encoDecoCell = tf.contrib.rnn.MultiRNNCell(
            [create_rnn_cell() for _ in range(self.args.numLayers)],
        )

        # Network input (placeholders)

        with tf.name_scope('placeholder_encoder'):
            self.encoderInputs  = [tf.placeholder(tf.int32,   [None, ]) for _ in range(self.args.maxLengthEnco)]  # Batch size * sequence length * input dim

        with tf.name_scope('placeholder_decoder'):
            self.decoderInputs  = [tf.placeholder(tf.int32,   [None, ], name='inputs') for _ in range(self.args.maxLengthDeco)]  # Same sentence length for input and output (Right ?)
            self.decoderTargets = [tf.placeholder(tf.int32,   [None, ], name='targets') for _ in range(self.args.maxLengthDeco)]
            self.decoderWeights = [tf.placeholder(tf.float32, [None, ], name='weights') for _ in range(self.args.maxLengthDeco)]

        # Define the network
        # Here we use an embedding model, it takes integer as input and convert them into word vector for
        # better word representation
        decoderOutputs, states = tf.contrib.legacy_seq2seq.embedding_rnn_seq2seq(
            self.encoderInputs,  # List<[batch=?, inputDim=1]>, list of size args.maxLength
            self.decoderInputs,  # For training, we force the correct output (feed_previous=False)
            encoDecoCell,
            self.textData.getVocabularySize(),
            self.textData.getVocabularySize(),  # Both encoder and decoder have the same number of class
            embedding_size=self.args.embeddingSize,  # Dimension of each word
            output_projection=outputProjection.getWeights() if outputProjection else None,
            feed_previous=bool(self.args.test)  # When we test (self.args.test), we use previous output as next input (feed_previous)
        )

        # TODO: When the LSTM hidden size is too big, we should project the LSTM output into a smaller space (4086 => 2046): Should speed up
        # training and reduce memory usage. Other solution, use sampling softmax

        # For testing only
        if self.args.test:
            if not outputProjection:
                self.outputs = decoderOutputs
            else:
                self.outputs = [outputProjection(output) for output in decoderOutputs]

            # TODO: Attach a summary to visualize the output

        # For training only
        else:
            # Finally, we define the loss function
            self.lossFct = tf.contrib.legacy_seq2seq.sequence_loss(
                decoderOutputs,
                self.decoderTargets,
                self.decoderWeights,
                self.textData.getVocabularySize(),
                softmax_loss_function= sampledSoftmax if outputProjection else None  # If None, use default SoftMax
            )
            tf.summary.scalar('loss', self.lossFct)  # Keep track of the cost

            # Initialize the optimizer
            opt = tf.train.AdamOptimizer(
                learning_rate=self.args.learningRate,
                beta1=0.9,
                beta2=0.999,
                epsilon=1e-08
            )
            self.optOp = opt.minimize(self.lossFct)

    def step(self, batch):
        """ Forward/training step operation.
        Does not perform run on itself but just return the operators to do so. Those have then to be run
        Args:
            batch (Batch): Input data on testing mode, input and target on output mode
        Return:
            (ops), dict: A tuple of the (training, loss) operators or (outputs,) in testing mode with the associated feed dictionary
        """

        # Feed the dictionary
        feedDict = {}
        ops = None

        if not self.args.test:  # Training
            for i in range(self.args.maxLengthEnco):
                feedDict[self.encoderInputs[i]]  = batch.encoderSeqs[i]
            for i in range(self.args.maxLengthDeco):
                feedDict[self.decoderInputs[i]]  = batch.decoderSeqs[i]
                feedDict[self.decoderTargets[i]] = batch.targetSeqs[i]
                feedDict[self.decoderWeights[i]] = batch.weights[i]

            ops = (self.optOp, self.lossFct)
        else:  # Testing (batchSize == 1)
            for i in range(self.args.maxLengthEnco):
                feedDict[self.encoderInputs[i]]  = batch.encoderSeqs[i]
            feedDict[self.decoderInputs[0]]  = [self.textData.goToken]

            ops = (self.outputs,)

        # Return one pass operator
        return ops, feedDict


# Using legacy_seq2seq

In [ ]:
import seq2seq_wrapper

model = seq2seq_wrapper.Seq2Seq(xseq_len=xseq_len,
                               yseq_len=yseq_len,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/twitter/',
                               emb_dim=emb_dim,
                               num_layers=3
                               )


val_batch_gen = data_utils.rand_batch_gen(validX, validY, 32)
train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)


sess = model.restore_last_session()
sess = model.train(train_batch_gen, val_batch_gen)


# using seq2seq library

In [4]:
sys.path.append('/home/dong/Dropbox/Projects/NLP/seq2seq')
from seq2seq.encoders import rnn_encoder

In [8]:
#super(UnidirectionalRNNEncoderTest, self).setUp()
#tf.logging.set_verbosity(tf.logging.INFO)
batch_size = 4
sequence_length = 16
input_depth = 10
mode = tf.contrib.learn.ModeKeys.TRAIN
params = rnn_encoder.UnidirectionalRNNEncoder.default_params()
params["rnn_cell"]["cell_params"]["num_units"] = 32
params["rnn_cell"]["cell_class"] = "BasicLSTMCell"
params

{'init_scale': 0.04,
 'rnn_cell': {'cell_class': 'BasicLSTMCell',
  'cell_params': {'num_units': 32},
  'dropout_input_keep_prob': 1.0,
  'dropout_output_keep_prob': 1.0,
  'num_layers': 1,
  'residual_combiner': 'add',
  'residual_connections': False,
  'residual_dense': False}}

In [ ]:
inputs = tf.random_normal(
    [self.batch_size, self.sequence_length, self.input_depth])
example_length = tf.ones(
    self.batch_size, dtype=tf.int32) * self.sequence_length
 
encode_fn = rnn_encoder.UnidirectionalRNNEncoder(self.params, self.mode)
encoder_output = encode_fn(inputs, example_length)
 
with self.test_session() as sess:
    sess.run(tf.global_variables_initializer())
    encoder_output_ = sess.run(encoder_output)
 
    np.testing.assert_array_equal(encoder_output_.outputs.shape,
                                  [self.batch_size, self.sequence_length, 32])
    self.assertIsInstance(encoder_output_.final_state,
                          tf.contrib.rnn.LSTMStateTuple)
    np.testing.assert_array_equal(encoder_output_.final_state.h.shape,
                                  [self.batch_size, 32])
    np.testing.assert_array_equal(encoder_output_.final_state.c.shape,
                                  [self.batch_size, 32])